In [1]:
import sdv
import pandas as pd
import numpy as np

In [42]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import OneHotEncoder

def evaluate_classifiers(synthetic_dataset, real_dataset, target_column):
    # Check if the dataset contains categorical variables
    if synthetic_dataset.select_dtypes(include=['object']).shape[1] > 0:
        # Concatenate the synthetic and real datasets
        combined_data = pd.concat([synthetic_dataset, real_dataset], ignore_index=True)

        # Apply one-hot encoding to handle categorical variables
        encoder = OneHotEncoder()
        combined_data_encoded = encoder.fit_transform(combined_data)

        # Separate the synthetic and real datasets
        new_data2_encoded = combined_data_encoded[:len(synthetic_dataset)]
        real_data_encoded = combined_data_encoded[len(synthetic_dataset):]

        synthetic_X = new_data2_encoded[:, :-1].toarray()
        synthetic_y = new_data2_encoded[:, -1].toarray()
        real_train_X, real_test_X, real_train_y, real_test_y = train_test_split(
            real_data_encoded[:, :-1].toarray(), real_data_encoded[:, -1].toarray(), test_size=0.2, random_state=42
        )
    else:
        synthetic_X = synthetic_dataset.drop(columns=[target_column]).to_numpy()
        synthetic_y = synthetic_dataset[target_column].to_numpy()
        real_train_X, real_test_X, real_train_y, real_test_y = train_test_split(
            real_dataset.drop(columns=[target_column]).to_numpy(), real_dataset[target_column].to_numpy(),
            test_size=0.2, random_state=42
        )

    # Train decision tree classifier on real_train dataset
    decision_tree_real = DecisionTreeClassifier()
    decision_tree_real.fit(real_train_X, real_train_y)
    decision_tree_pred_real = decision_tree_real.predict(real_test_X)
    f1_score_decision_tree_real = f1_score(real_test_y, decision_tree_pred_real)

    # Train decision tree classifier on synthetic dataset
    decision_tree_synthetic = DecisionTreeClassifier()
    decision_tree_synthetic.fit(synthetic_X, synthetic_y)
    decision_tree_pred_synthetic = decision_tree_synthetic.predict(real_test_X)
    f1_score_decision_tree_synthetic = f1_score(real_test_y, decision_tree_pred_synthetic)

    # Train gradient boost classifier on real_train dataset
    gradient_boost_real = GradientBoostingClassifier()
    gradient_boost_real.fit(real_train_X, real_train_y)
    gradient_boost_pred_real = gradient_boost_real.predict(real_test_X)
    f1_score_gradient_boost_real = f1_score(real_test_y, gradient_boost_pred_real)

    # Train gradient boost classifier on synthetic dataset
    gradient_boost_synthetic = GradientBoostingClassifier()
    gradient_boost_synthetic.fit(synthetic_X, synthetic_y)
    gradient_boost_pred_synthetic = gradient_boost_synthetic.predict(real_test_X)
    f1_score_gradient_boost_synthetic = f1_score(real_test_y, gradient_boost_pred_synthetic)

    # Create a dictionary to store the F1-scores
    f1_scores = {
        'Decision Tree (Real)': f1_score_decision_tree_real,
        'Decision Tree (Synthetic)': f1_score_decision_tree_synthetic,
        'Gradient Boosting (Real)': f1_score_gradient_boost_real,
        'Gradient Boosting (Synthetic)': f1_score_gradient_boost_synthetic,
    }

    return f1_scores

# Example usage:
# Assuming your synthetic dataset is stored in the variable 'new_data


In [43]:
real_data=pd.read_csv('german_credit_data.csv')
new_data2=pd.read_csv('new_data2.csv')

In [ ]:
# List of categorical columns to be one-hot encoded
categorical_columns = ['Sex', 'Job', 'Housing', 'Saving accounts', 'Checking account', 'Purpose', 'Risk']

# Perform one-hot encoding
df_encoded = pd.get_dummies(df_constant, columns=categorical_columns)
